Things to do:

1. build data loader with augmentation
2. CNN architecture
   - transfer learning
3. Train
4. inference

In [2]:
!pip install torchvision
!pip install scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 110.4 MB/s eta 0:00:0000:01


In [3]:
import os
import shutil

import pandas as pd
import numpy as np

from skimage import io

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Resize,  RandomHorizontalFlip, Normalize, Compose
#from torchvision.transforms import ToTensor, RandomSizedCrop, RandomHorizontalFlip, Normalize


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [8]:
!pwd

/root/data/image_classification/notebooks


In [4]:
os.chdir("/root/data/image_classification/")

In [5]:
test_path = "./data/test1"
train_path = "./data/train"
val_path = "./data/val"
training_path = "./data/training"


In [15]:
if not os.path.exists(val_path):
    os.makedirs(val_path)
    
if not os.path.exists(training_path):
    os.makedirs(training_path)


In [17]:
# Split train to training and val set:

files = []
label = []

# Iterate directory
for path in os.listdir(train_path):
    # check if current path is a file
    files.append(path)
    if "cat" in path:
        label.append("cat")
    else:
        label.append("dog")
    


In [96]:
# Create test Metafile

files = []
label = []

# Iterate directory
for path in os.listdir(test_path):
    # check if current path is a file
    files.append(path)
    if "cat" in path:
        label.append("cat")
    else:
        label.append("dog")
    

In [99]:
test_meta_df = pd.DataFrame({'files':files, "label":label})
test_meta_df['path'] = test_path + "/"+ test_meta_df['files']


In [100]:
test_meta_df.head()

,files,label,path
0,10592.jpg,dog,./data/test1/10592.jpg
1,7217.jpg,dog,./data/test1/7217.jpg
2,3653.jpg,dog,./data/test1/3653.jpg
3,4382.jpg,dog,./data/test1/4382.jpg
4,2924.jpg,dog,./data/test1/2924.jpg


In [33]:
train_meta_df = pd.DataFrame({'files':files, "label":label})

# Shuffle
train_meta_df = train_meta_df.sample(frac=1)

train_split = train_meta_df[:int(len(train_meta_df)*.8)]
val_split = train_meta_df[int(len(train_meta_df)*.8):]

## Split to Train/Val

In [49]:
for i, row in val_split.iterrows():
    #print(row.files)
    shutil.copyfile(f"{train_path}/{row.files}", f"{val_path}/{row.files}")

print("complete")

complete


In [50]:
for i, row in train_split.iterrows():
    #print(row.files)
    shutil.copyfile(f"{train_path}/{row.files}", f"{training_path}/{row.files}")

print("complete")

complete


In [68]:
train_split.reset_index(inplace = True)
val_split.reset_index(inplace = True)

In [69]:
train_split.drop(columns = ['index'], inplace = True)
val_split.drop(columns = ['index'], inplace = True)


/tmp/ipykernel_881/2520651009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_split.drop(columns = ['index'], inplace = True)
/tmp/ipykernel_881/2520651009.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_split.drop(columns = ['index'], inplace = True)


In [71]:
training_path

'./data/training'

In [66]:
train_split['path'] = training_path + "/"+ train_split['files']
val_split['path'] = val_path + "/"+ val_split['files']

/tmp/ipykernel_881/1925462727.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_split['path'] = training_path + "/"+ train_split['files']
/tmp/ipykernel_881/1925462727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_split['path'] = val_path + "/"+ val_split['files']


In [101]:
train_split.to_csv("./data/training_meta.csv", index = False)
val_split.to_csv("./data/val_meta.csv", index = False)
test_meta_df.to_csv("./data/test_meta.csv", index = False)

In [ ]:
## Copy to S3: Run commands on terminal
#aws s3 cp ./training s3://sagemaker-us-east-1-175218665739/dog_cat_images/train --recursive
#aws s3 cp ./test1 s3://sagemaker-us-east-1-175218665739/dog_cat_images/test --recursive
#aws s3 cp ./test_meta.csv ./training_meta.csv ./val_meta.csv s3://sagemaker-us-east-1-175218665739/dog_cat_images



## Research and References

# Competition:
https://www.kaggle.com/competitions/dogs-vs-cats/code

Articles: 
1. https://www.analyticsvidhya.com/blog/2021/06/transfer-learning-using-vgg16-in-pytorch/
2. https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

Documentation:
1. https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html
2. https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
3. https://pytorch.org/vision/main/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py